<a href="https://colab.research.google.com/github/LarryJilesJr/Home_Sales/blob/main/My_Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install findspark
!pip install pyspark
!pip install pyarrow
!pip install fastparquet
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=166c8ea79104ab2cecf9b03ea2668d6e4ed8237a52de727c4c1f07d520a34534
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.3 MB/s eta 0:00:00


In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.co

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_revised_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_revised_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_revised_df.createOrReplaceTempView("home_sales_temp_view")

In [50]:
# Describing the DataFrame
home_sales_revised_df.describe()

DataFrame[summary: string, id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [40]:
# Showing schema for the DataFrame
home_sales_revised_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Query to calculate average price for a four-bedroom house sold in each year
average_price_query = """
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_temp_view
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year DESC
"""

# Execute the query
average_price_df = spark.sql(average_price_query)

# Show the DataFrame
average_price_df.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built
# that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# Query to calculate average price for homes with 3 bedrooms and 3 bathrooms for each year built
average_price_query = """
    SELECT date_built, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_temp_view
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built DESC
"""

# Execute the query
average_price_df = spark.sql(average_price_query)

# Show the DataFrame
average_price_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    292676.79|
|      2016|    290555.07|
|      2015|     288770.3|
|      2014|    290852.27|
|      2013|    295962.27|
|      2012|    293683.19|
|      2011|    291117.47|
|      2010|    292859.62|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# Query to calculate average price for homes with specific criteria for each year built
average_price_query = """
    SELECT date_built, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_temp_view
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built DESC
"""

# Execute the query
average_price_df = spark.sql(average_price_query)

# Show the DataFrame
average_price_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Import required libraries
from pyspark.sql.functions import lit

# Record the start time
start_time = time.time()

# Query to calculate the average price of homes greater than or equal to $350,000 and group by the "view" rating
average_price_query = """
    SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
    FROM home_sales_temp_view
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
"""

# Execute the query
average_price_df = spark.sql(average_price_query)

# Generate a DataFrame with view ratings from 80 to 99
view_ratings_df = spark.range(80, 100).withColumnRenamed("id", "view")

# Left join the generated view ratings DataFrame with your average_price_df
output_df = view_ratings_df.join(average_price_df, "view", "left").fillna(0)

# Sort output_df in descending order by view rating
output_df = output_df.orderBy("view", ascending=False)

# Show the DataFrame
output_df.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+

--- 1.7383694648742676 seconds ---


In [12]:
# 7. Cache the temporary table home_sales
spark.catalog.cacheTable("home_sales_temp_view")

In [13]:
# 8. Check if the table is cached
table_is_cached = spark.catalog.isCached("home_sales_temp_view")

# Print the result
if table_is_cached:
    print("The table 'home_sales_temp_view' is cached.")
else:
    print("The table 'home_sales_temp_view' is not cached.")


The table 'home_sales_temp_view' is cached.


In [29]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Record the start time
cached_start_time = time.time()

# Execute the query using the cached data and filter out the view ratings with an average price greater than or equal to $350,000
filtered_query = """
    SELECT *
    FROM home_sales_temp_view
    WHERE view NOT IN (
        SELECT view
        FROM home_sales_temp_view
        WHERE price >= 350000
        GROUP BY view
    )
"""

# Execute the query
filtered_df = spark.sql(filtered_query)

# Generate a DataFrame with view ratings from 80 to 99
view_ratings_df = spark.range(80, 100).withColumnRenamed("id", "view")

# Left join the generated view ratings DataFrame with your average_price_df
cached_output_df = view_ratings_df.join(average_price_df, "view", "left").fillna(0)

# Sort output_df in descending order by view rating
cached_output_df = cached_output_df.orderBy("view", ascending=False)

# Show the DataFrame
cached_output_df.show()

# Create a variable for calculated cached_runtime
cached_runtime = "The cached query" + "--- %s seconds ---" % (time.time() - cached_start_time)

# Print the runtime
print(cached_runtime)

# Record the start time
start_time = time.time()

# Query to calculate the average price of homes greater than or equal to $350,000 and group by the "view" rating
average_price_query = """
    SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
    FROM home_sales_temp_view
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
"""

# Execute the query
average_price_df = spark.sql(average_price_query)

# Generate a DataFrame with view ratings from 80 to 99
view_ratings_df = spark.range(80, 100).withColumnRenamed("id", "view")

# Left join the generated view ratings DataFrame with your average_price_df
output_df = view_ratings_df.join(average_price_df, "view", "left").fillna(0)

# Sort output_df in descending order by view rating
output_df = output_df.orderBy("view", ascending=False)

# Show the DataFrame
output_df.show()

# Create a variable for calculated cached_runtime
uncached_runtime = "The uncached query" + "--- %s seconds ---" % (time.time() - start_time)

# Calculate and print the runtime
print(uncached_runtime)

# Compare the runtimes and print a message
if cached_runtime < uncached_runtime:
    print("Cached query ran faster.")
elif cached_runtime > uncached_runtime:
    print("Uncached query ran faster.")
else:
    print("Both queries ran at the same speed.")

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+

The cached query--- 0.7905855178833008 seconds ---
+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  

In [44]:
# 10. Partition the home sales dataset by date_built field.
home_sales_revised_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")


In [45]:
# 11. Read the Parquet-formatted data
parquet_data_df = spark.read.parquet("home_sales_partitioned")

# Show the schema and some sample data
parquet_data_df.printSchema()
parquet_data_df.show()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3| 

In [46]:
# 12. Create a temporary table for the parquet data.
parquet_data_df.createOrReplaceTempView('parquet_home_sales_data')

In [47]:
# Describing the DataFrame
parquet_data_df.describe()

DataFrame[summary: string, id: string, date: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, date_built: string]

In [48]:
# Showing schema for the DataFrame
parquet_data_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- date_built: integer (nullable = true)



In [51]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Record the start time
parquet_start_time = time.time()

# Execute the query using the cached data and filter out the view ratings with an average price greater than or equal to $350,000
parquet_filtered_query = """
    SELECT *
    FROM parquet_home_sales_data
    WHERE view NOT IN (
        SELECT view
        FROM parquet_home_sales_data
        WHERE price >= 350000
        GROUP BY view
    )
"""

# Execute the query
parquet_filtered_df = spark.sql(parquet_filtered_query)

# Generate a DataFrame with view ratings from 80 to 99
view_ratings_df = spark.range(80, 100).withColumnRenamed("id", "view")

# Left join the generated view ratings DataFrame with your average_price_df
parquet_output_df = view_ratings_df.join(average_price_df, "view", "left").fillna(0)

# Sort output_df in descending order by view rating
parquet_output_df = parquet_output_df.orderBy("view", ascending=False)

# Show the DataFrame
parquet_output_df.show()

# Create a variable for calculated cached_runtime
parquet_runtime = "The parquet query" + "--- %s seconds ---" % (time.time() - parquet_start_time)

# Print the runtime
print(parquet_runtime)

# Record the start time
cached_start_time = time.time()

# Execute the query using the cached data and filter out the view ratings with an average price greater than or equal to $350,000
filtered_query = """
    SELECT *
    FROM home_sales_temp_view
    WHERE view NOT IN (
        SELECT view
        FROM home_sales_temp_view
        WHERE price >= 350000
        GROUP BY view
    )
"""

# Execute the query
filtered_df = spark.sql(filtered_query)

# Generate a DataFrame with view ratings from 80 to 99
view_ratings_df = spark.range(80, 100).withColumnRenamed("id", "view")

# Left join the generated view ratings DataFrame with your average_price_df
cached_output_df = view_ratings_df.join(average_price_df, "view", "left").fillna(0)

# Sort output_df in descending order by view rating
cached_output_df = cached_output_df.orderBy("view", ascending=False)

# Show the DataFrame
cached_output_df.show()

# Create a variable for calculated cached_runtime
cached_runtime = "The cached query" + "--- %s seconds ---" % (time.time() - cached_start_time)

# Print the runtime
print(cached_runtime)

# Compare the runtimes and print a message
if parquet_runtime < cached_runtime:
    print("Parquet query ran faster.")
elif parquet_runtime > cached_runtime:
    print("Cached query ran faster.")
else:
    print("Both queries ran at the same speed.")

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+

The parquet query--- 0.8513364791870117 seconds ---
+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
| 

In [52]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales_temp_view")

In [53]:
# 15. Check if the home_sales is no longer cached
# Check if the table is uncached
table_is_uncached = not spark.catalog.isCached("home_sales_temp_view")

if table_is_uncached:
    print("The table home_sales_temp_view is uncached.")
else:
    print("The table home_sales_temp_view is cached.")


The table home_sales_temp_view is uncached.
